<a href="https://colab.research.google.com/github/da0ran9e/ArtificialIntelligent/blob/main/Image_to_3D_by_3DBullet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



*This* Google Collab is made possible by PIFuHD reaserch paper: https://
shunsukesaito.github.io/PIFuHD/

Make Sure you have GPU selected as your default accelerator.



Code Merged,Updated,Fixed and Simplified by 3DBullet


---



**Only run the first cell once after opening this Google Collab file.**

In [ ]:
#@title Downloading necessary dependencies
#Code Merged,Updated,Fixed and Simplified by 3DBullet
#Configure Python version
!apt-get update -y
!apt-get install python3.8
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!update-alternatives --set python3 /usr/bin/python3.8
!python3 --version
!apt-get update
!apt-get install python3-pip
!sudo apt upgrade

#Clone 
!git clone https://github.com/facebookresearch/pifuhd
%cd pifuhd
!python setup.py install

#Download all necessary dependencies
!python3.8 -m pip install 'torch==1.6.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
!python3.8 -m pip install 'torchvision==0.7.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
!python3.8 -m pip install 'pytorch3d==0.2.5'
!python3.8 -m pip install models
!python3.8 -m pip install scikit-image==0.18.3
!python3.8 -m pip install opencv-python
!python3.8 -m pip install numpy==1.19.1
!python3.8 -m pip install matplotlib==3.3.4
!python3.8 -m pip install ipykernel

#Downloading all necessary files for cropping the image

%cd /content/pifuhd/
!sh ./scripts/download_trained_model.sh

image_path = '/content/pifuhd/sample_images/sample_111'
#Key: M0RCdWxsZXQgTWFkZSB0aGlz


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [76.4 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [993 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/m



---



---

You can run the next cell as many times as you desire.

**Only .png and .jpg files are supported currently.**

After running the next cell you need to select the desired image from your machine


---



---



In [ ]:
#@title Image to 3D mesh
#Code Merged,Updated,Fixed and Simplified by 3DBullet


#Delete unnecissary files from previous run

import os

from google.colab import files
if os.path.isfile(image_path):
  if os.path.isfile('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name):
    os.remove('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name)
if os.path.isfile(image_path):
  if os.path.isfile('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name):
    os.remove('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name)
if os.path.isfile(image_path):
  if os.path.isfile('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name):
    os.remove('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name)
if os.path.isfile(image_path):
  if os.path.isfile('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name):
    os.remove(image_path)

#Ask for an input image from user and store it
%cd /content/pifuhd/sample_images

from google.colab import files

filename = list(files.upload().keys())[0]

image_path = '/content/pifuhd/sample_images/%s' % filename

image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# output paths
obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = '/content/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

#download neccissary files for cropping the input image.

%cd /content
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git
%cd /content/lightweight-human-pose-estimation.pytorch/
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

#Process that crops the image
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo
def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

#Starting from previously determened checkpoint.
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)
get_rect(net.cuda(), [image_path], 512)

#Normal map and 3D Mesh generation using the paper pre-trained model.
%cd /content/pifuhd/
# Warning: all images with the corresponding rectangle files under -i will be processed. 
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

# seems that 256 is the maximum resolution that can fit into Google Colab. 
# If you want to reconstruct a higher-resolution mesh, please try with your own machine. 


#Download Files
from google.colab import files
files.download('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name)
files.download('/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name)

#import sys

#sys.path.append('/usr/local/lib/python3.8/dist-packages')

import pytorch3d

#Display the preview of the 3D Model.
from lib.colab_util import generate_video_from_obj, set_renderer, video

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# we cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)

#Key: M0RCdWxsZXQgRml4ZWQgVGhpcw==


/content/pifuhd/sample_images


Saving depositphotos_97236866-stock-photo-handsome-sexy-male-model-posing.jpg to depositphotos_97236866-stock-photo-handsome-sexy-male-model-posing (1).jpg
/content
fatal: destination path 'lightweight-human-pose-estimation.pytorch' already exists and is not an empty directory.
/content/lightweight-human-pose-estimation.pytorch
--2023-04-23 11:17:38--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 104.83.239.160, 2600:1408:c400:395::4b21, 2600:1408:c400:391::4b21
Connecting to download.01.org (download.01.org)|104.83.239.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87959810 (84M) [application/octet-stream]
Saving to: ‘checkpoint_iter_370000.pth.1’

checkpoint_iter_370 100%[===================>]  83.88M  51.7MB/s    in 1.6s    

2023-04-23 11:17:41 (51.7 MB/s) - ‘checkpoint_iter_370000.pth.1’ saved [87959810/87959810]



<ipython-input-6-d05da0d401fc>:100: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)


/content/pifuhd
/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Resuming from  ./checkpoints/pifuhd.pt
test data size:  1
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/1 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_depositphotos_97236866-stock-photo-handsome-sexy-male-model-posing_256.obj
100% 1/1 [00:07<00:00,  7.00s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ImportError: ignored